2- Escolha um conjunto de textos  (poemas, letras de música, etc )

a) Separe 10% dos textos como conjunto de testes e e o restante como conjunto de treinamento

b) Construa um modelo de linguagen  explorando diferentes n-gramas (unigrama, bigrama e trigrama) e métodos de suavização (Laplace vs Lidstone)

c) Faça uma análise do melhor modelo de linguagem com base na sua perplexidade no conjunto de teste

d) Escolha o melhor modelo e gere um texto de até 30 tokens predizendo a proxima palavra de maxima probabilidade.

In [1]:
import nltk
from nltk import bigrams, trigrams
from nltk.lm import MLE, Laplace, Lidstone
from nltk.lm.preprocessing import pad_both_ends
import re
import random
import string
from nltk.lm.preprocessing import padded_everygram_pipeline
nltk.download('stopwords')
nltk.download('punkt')

def pre_processamento(texto):

    # seleciona apenas letras e coloca todas em minúsculo
    letras_min =  re.findall(r'\b[A-zÀ-úü]+\b', texto.lower())

    # remove stopwords
    stopwords = nltk.corpus.stopwords.words('portuguese')
    stop = set(stopwords)
    sem_stopwords = [w for w in letras_min if w not in string.punctuation]


    # juntando os tokens novamente em formato de texto
    texto_limpo = " ".join(sem_stopwords)

    return texto_limpo

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [2]:
# id do corpus
# no nosso caso estamos usando id machado
nltk_id = 'machado'

# eh necessario baixar o corpus
nltk.download(nltk_id)

# agora o corpus esta acessivel
# visualizando as obras disponiveis

#print(nltk.corpus.machado.readme())
# ou

#print(nltk.corpus.machado.fileids())

# apos escolher a obra
# salvamos a string em uma variavel
ressurreicao = pre_processamento(nltk.corpus.machado.raw('romance/marm01.txt'))
mao_luva = pre_processamento(nltk.corpus.machado.raw('romance/marm02.txt'))
helena = pre_processamento(nltk.corpus.machado.raw('romance/marm03.txt'))
iaiá_garcia = pre_processamento(nltk.corpus.machado.raw('romance/marm04.txt'))
memorias_postumas= pre_processamento(nltk.corpus.machado.raw('romance/marm05.txt'))
casa_velha = pre_processamento(nltk.corpus.machado.raw('romance/marm06.txt'))
quincas_borba = pre_processamento(nltk.corpus.machado.raw('romance/marm07.txt'))
dom_casmurro = pre_processamento(nltk.corpus.machado.raw('romance/marm08.txt'))
esau_jaco = pre_processamento(nltk.corpus.machado.raw('romance/marm09.txt'))
memorial_aires = pre_processamento(nltk.corpus.machado.raw('romance/marm10.txt'))



[nltk_data] Downloading package machado to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...


```
# 1. paddedLine = [list(pad_both_ends(nltk.word_tokenize(texto), n=2))]
# 2. train, vocab = padded_everygram_pipeline(2, paddedLine)
# 3. lmMLE = MLE(2)
# 4. lmLAP = Laplace(2)
# 5. lmLIDS = Lidstone(0.1, 2)
# 6. lmMLE.fit(train, vocabulary_text=vocab)
# 7. lmLAP.fit(train, vocabulary_text=vocab)
# 8. lmLIDS.fit(train, vocabulary_text=vocab)
```
 A linha 1 em questão tokeniza o texto em palavras, adiciona tokens < s> e < /s> de início e fim, criando bigramas apropriados, e coloca o resultado em uma lista.


 A linha 2 em questão cria o conjunto de treinamento e o vocabulário necessários para treinar o modelo de linguagem com bigramas (n-gramas de tamanho 2)


 Nas linhas 3,4,5,6 é realizado em sequência a criação dos modelos de linguagens(MLE) de bigramas, modelo de linguagem com suavização Laplace(Laplace(2)) de bigrama e o modelo de linguagem com suavização Lidstone(Lidstone(0.1,2)) (Todos são bigramas)


Na linha seguinte é feito o treinamento dos modelos.

In [3]:
# Exemplo de texto
texto = pre_processamento(nltk.corpus.machado.raw('romance/marm01.txt'))

indice_90_percent = int(0.9 * len(texto))

texto_90_percent = nltk.word_tokenize(texto[:indice_90_percent])
texto_10_percent = nltk.word_tokenize(texto[indice_90_percent:])


paddedLine = [list(pad_both_ends(texto_90_percent, n=2))]


trainMLE, vocabMLE = padded_everygram_pipeline(3, paddedLine)
trainLAP, vocabLAP = padded_everygram_pipeline(3, paddedLine)
trainLIDS, vocabLIDS = padded_everygram_pipeline(3, paddedLine)

lmMLE = MLE(3)
lmLAP = Laplace(3)
lmLIDS = Lidstone(0.1,3)

lmLIDS.fit(trainLIDS, vocabulary_text=vocabLIDS)
lmLAP.fit(trainLAP, vocabulary_text=vocabLAP)
lmMLE.fit(trainMLE, vocabulary_text=vocabMLE)




In [4]:
trigramas_test = [list(trigrams(pad_both_ends(texto_10_percent, n=2)))]

In [5]:
print(lmLIDS.perplexity(trigramas_test))
print(lmLIDS.generate(30))

5561.000000000008
['que', 'não', 'podia', 'deixar', 'de', 'sorrir', 'só', 'havia', 'uma', 'inexplicável', 'melancolia', 'anterior', 'à', 'sua', 'cabeça', 'angélica', 'não', 'pensava', 'nele', 'o', 'comentário', 'das', 'palavras', 'que', 'houve', 'entre', 'ambas', 'quando', 'a', 'mucama']


In [6]:
print(lmLAP.perplexity(trigramas_test))
print(lmLAP.generate(30))

5561.000000000008
['rapariga', 'ao', 'menos', 'parte', 'dele', 'que', 'era', 'ela', 'cansada', 'de', 'esperar', 'que', 'lhe', 'desse', 'tempo', 'o', 'amor', 'nos', 'unisse', 'meneses', 'não', 'estava', 'para', 'estas', 'viagens', 'contínuas', 'e', 'eu', 'sentiria', 'se']


In [7]:
print(lmMLE.perplexity(trigramas_test))
print(lmMLE.generate(30))

inf
['conversa', 'portanto', 'não', 'foi', 'só', 'o', 'que', 'era', 'o', 'desfecho', 'lógico', 'e', 'igual', 'a', 'si', 'próprio', 'a', 'execução', 'seguiu', 'de', 'perto', 'a', 'idéia', 'de', 'que', 'era', 'a', 'transformação', 'de', 'sua']
